# Conversational UI Chatbot App with Gemini, LangChain and Streamlit

Here we will build a advanced Conversational UI-based chatbot using LangChain and Streamlit with the following features:

- Custom Landing Page
- Conversational memory

## Install App and LLM dependencies

In [22]:
!pip install langchain==0.1.12 -q
!pip install langchain-google-genai==0.0.7 -q
!pip install langchain-community==0.0.29 -q
!pip install streamlit==1.32.2 -q
!pip install pyngrok==7.1.5 -q
!pip install google-generativeai>=0.3.2 -q

## Load Gemini API Credentials

Here we load it from a file so we don't explore the credentials on the internet by mistake

In [23]:
import os
from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GEMINI_API_KEY')

## Basic Streamlit UI without Memory

This simple version shows how to:

1. Create a basic Streamlit interface
2. Connect directly to Gemini
3. Process basic Q&A without memory

In [24]:
# %%writefile kjsit_bot.py

import streamlit as st
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

st.title("🤖 KJSIT Assistant Bot")

# Initialize the LLM
gemini = ChatGoogleGenerativeAI(model='gemini-1.5-flash',
                                temperature=0.2)

# System prompt about KJSIT
system_prompt = """
You are an Admission Assistant for K J Somaiya Institute of Technology (KJSIT), Mumbai.

Your job is to provide detailed and accurate information regarding:
- B.Tech and M.Tech admissions
- Specializations (AI & DS, Computer Engineering, IT, EXTC)
- Eligibility criteria and cutoffs
- Fees and application deadlines
- Placements, highest and average packages
- Internship support
- Faculty and infrastructure
- Research and Ph.D. opportunities
- Campus life, clubs, events and hostels

Only respond with factual and official information from https://kjsit.somaiya.edu.in/en. If you don’t know something, politely direct users to the official website or contact email.
Be friendly, concise, and helpful.
"""

# Create prompt template
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    HumanMessagePromptTemplate.from_template("{input}")
])

# Chat interaction
if user_input := st.chat_input("Ask me anything about KJSIT Admissions!"):
    st.chat_message("user").write(user_input)

    with st.chat_message("assistant"):
        chain = prompt | gemini
        response = chain.invoke({"input": user_input})
        st.write(response.content)


## Start the app

In [25]:
!streamlit run kjsit_bot.py --server.port=8989 &>./logs.txt &

In [26]:
from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()

ngrok.set_auth_token(userdata.get('NGROK_API_KEY'))

# Open an HTTPs tunnel on port XXXX which you get from your `logs.txt` file
ngrok_tunnel = ngrok.connect(8989)
print("Streamlit App:", ngrok_tunnel.public_url)

Streamlit App: https://f660-34-86-224-40.ngrok-free.app


## Remove running app processes

In [27]:
ngrok.kill()

In [28]:
!ps -ef | grep streamlit

root        1660       1  2 08:04 ?        00:00:01 /usr/bin/python3 /usr/local/bin/streamlit run manual_app.py --server.port=8989
root        2007    1182  0 08:05 ?        00:00:00 /bin/bash -c ps -ef | grep streamlit
root        2009    2007  0 08:05 ?        00:00:00 grep streamlit


In [29]:
!sudo kill -9  6590

kill: (6590): No such process


## Part 2: Showing Messages with Manual Memory

This demonstrates:

1. How to manually implement memory in Streamlit
2. The challenge of formatting context for the LLM
3. Why specialized tools like LangChain help

In [30]:
%%writefile manual_app.py

import streamlit as st
from langchain_google_genai import ChatGoogleGenerativeAI

st.title("AI Assistant with Manual Memory")

# Initialize the LLM
gemini = ChatGoogleGenerativeAI(model='gemini-2.0-flash-001',
                               temperature=0.1)

# Initialize session state for memory
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display existing messages
for message in st.session_state.messages:
    st.chat_message(message["role"]).write(message["content"])

# Process new input
if user_input := st.chat_input("Ask a question"):
    # Add user message to history
    st.session_state.messages.append({"role": "human", "content": user_input})
    st.chat_message("human").write(user_input)

    # Manually create a context string from history
    context = "Previous conversation:\n"
    for msg in st.session_state.messages:
        context += f"{msg['role']}: {msg['content']}\n"

    with st.chat_message("ai"):
        # Send context + new question
        full_prompt = context + "\nPlease respond to the last question."
        response = gemini.invoke(full_prompt)
        st.write(response.content)

        # Add AI response to history
        st.session_state.messages.append({"role": "ai", "content": response.content})

Overwriting manual_app.py


In [31]:
!streamlit run manual_app.py --server.port=8989 &>./logs.txt &

In [32]:


from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()

ngrok.set_auth_token(userdata.get('NGROK_API_KEY'))

# Open an HTTPs tunnel on port XXXX which you get from your `logs.txt` file
ngrok_tunnel = ngrok.connect(8989)
print("Streamlit App:", ngrok_tunnel.public_url)

Streamlit App: https://5c67-34-86-224-40.ngrok-free.app


In [33]:

ngrok.kill()

In [34]:
!ps -ef | grep streamlit

root        1660       1  2 08:04 ?        00:00:01 /usr/bin/python3 /usr/local/bin/streamlit run manual_app.py --server.port=8989
root        2046    1182  0 08:05 ?        00:00:00 /bin/bash -c ps -ef | grep streamlit
root        2048    2046  0 08:05 ?        00:00:00 grep streamlit


In [35]:
!sudo kill -9 4087

kill: (4087): No such process


## Part 3: Full LangChain Integration

In [36]:
%%writefile langchain_app.py

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from operator import itemgetter
import streamlit as st

# Customize initial app landing page
st.set_page_config(page_title="AI Assistant", page_icon="🤖")
st.title("Welcome I am AI Assistant 🤖")

# Load a connection to Gemini LLM
gemini = ChatGoogleGenerativeAI(model='gemini-2.0-flash-001',
                               temperature=0.1,
                               convert_system_message_to_human=True)

# Add a basic system prompt for LLM behavior
SYS_PROMPT = """
              Act as a helpful assistant and answer questions to the best of your ability.
              Do not make up answers.
              """

# Create a prompt template for langchain to use history to answer user prompts
prompt = ChatPromptTemplate.from_messages(
  [
    ("system", SYS_PROMPT),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
  ]
)

# Create a basic llm chain
llm_chain = (
  prompt
  |
  gemini
)

# Store conversation history in Streamlit session state
streamlit_msg_history = StreamlitChatMessageHistory()

# Create a conversation chain
conversation_chain = RunnableWithMessageHistory(
  llm_chain,
  lambda session_id: streamlit_msg_history,  # Accesses memory
  input_messages_key="input",
  history_messages_key="history",
)

# Render current messages from StreamlitChatMessageHistory
for msg in streamlit_msg_history.messages:
  st.chat_message(msg.type).write(msg.content)

# If user inputs a new prompt, display it and show the response
if user_prompt := st.chat_input():
  st.chat_message("human").write(user_prompt)
  # This is where response from the LLM is shown
  with st.chat_message("ai"):
    config = {"configurable": {"session_id": "any"}}
    # Get llm response
    response = conversation_chain.invoke({"input": user_prompt}, config)
    st.markdown(response.content) # Display response directly

Overwriting langchain_app.py


## Start the app

In [37]:
!streamlit run langchain_app.py --server.port=8989 &>./logs.txt &

In [38]:
from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()

ngrok.set_auth_token(userdata.get('NGROK_API_KEY'))

# Open an HTTPs tunnel on port XXXX which you get from your `logs.txt` file
ngrok_tunnel = ngrok.connect(8989)
print("Streamlit App:", ngrok_tunnel.public_url)

Streamlit App: https://6484-34-86-224-40.ngrok-free.app


## Remove running app processes